<a href="https://colab.research.google.com/github/dungdt-infopstats/TV-command-synthesis/blob/main/notebooks/WER_COMPUTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

tridungdo_ddss_aug_ratio_test_path = kagglehub.dataset_download('tridungdo/ddss-aug-ratio-test')
tridungdo_res_analysis_test_res_json_path = kagglehub.dataset_download('tridungdo/res-analysis-test-res-json')

print('Data source import complete.')


In [ ]:
!gdown 10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk

Downloading...
From (original): https://drive.google.com/uc?id=10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk
From (redirected): https://drive.google.com/uc?id=10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk&confirm=t&uuid=47472f0c-c125-41d6-af66-f448d9cb18ff
To: /kaggle/working/synthesis.zip
100%|████████████████████████████████████████| 955M/955M [00:19<00:00, 49.0MB/s]


In [ ]:
!unzip -q synthesis.zip

In [ ]:
import os

for file in os.listdir('synthesis'):
    path = f"synthesis/{file}"
    !unzip -q {path} -d command_text

In [ ]:
import json
import pandas as pd
list_row = []
for folder in os.listdir('command_text'):
    for file in os.listdir(os.path.join('command_text', folder)):
        if file.endswith('.json'):
            with open(os.path.join('command_text', folder, file)) as f:
                data = json.load(f)
                id_f = folder
                data['id'] = id_f
                list_row.append(data)


df_text = pd.DataFrame(list_row)

In [ ]:
df_text['text_active'] = None

In [ ]:
for idx, row in df_text.iterrows():
    if row['type'] == 'non_active':
        df_text.loc[idx, 'text_active'] = None
    elif row['num_segments'] == 0:
        df_text.loc[idx, 'text_active'] = row['command']
    else:
        active_list = []
        for i in range(row['num_segments']):
            if row['type_segments'][str(i)] == 'active':
                active_list.append(row['text_segments'][str(i)])

        text = " ".join(active_list)
        df_text.loc[idx, 'text_active'] = text


In [ ]:
import json

with open("/kaggle/input/res-analysis-test-res-json/res_analysis.json") as f:
    data = json.load(f)

In [ ]:
data_analysis = pd.DataFrame(data)

In [ ]:
data_analysis['id_f'] = data_analysis['id'].str.rsplit(pat="_", n=1).str[0]

In [ ]:
merge = pd.merge(data_analysis, df_text, left_on = 'id_f', right_on = 'id')

In [ ]:
!pip install faster-whisper
!pip install pydub
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.0 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 21.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 29.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 34.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.3 MB/s eta 0:00:0000:0100:01


In [ ]:
import pandas as pd
import numpy as np
import os
import tempfile
from typing import List, Dict, Any, Optional, Tuple
from pydub import AudioSegment
import jiwer
from faster_whisper import WhisperModel, BatchedInferencePipeline
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading
from functools import partial
import ast

# ==========================================================
# Helper: Convert vector -> time ranges
# ==========================================================
def vector_to_time_ranges(
    vector: List[int],
    frame_dur: float,
    hop_dur: float,
    pad: int = 1
) -> List[Dict[str, float]]:
    """
    Convert binary vector (0/1) -> list of {"start", "end"} in seconds.
    Merge segments when short gaps exist (controlled by pad).
    """
    ranges = []
    n = len(vector)
    i = 0

    while i < n:
        if vector[i] == 1:
            start = i * hop_dur
            j = i
            while j < n and any(vector[max(0, j - pad):min(n, j + pad + 1)]):
                j += 1
            end = j * hop_dur + frame_dur
            ranges.append({"start": start, "end": end})
            i = j
        else:
            i += 1
    return ranges


# ==========================================================
# Helper: Load audio and slice
# ==========================================================
def load_audio_as_pydub(path: str, sr: Optional[int] = None) -> AudioSegment:
    """Load audio file using pydub with optional sample rate conversion."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Audio file not found: {path}")

    audio = AudioSegment.from_file(path)
    if sr is not None and audio.frame_rate != sr:
        audio = audio.set_frame_rate(sr)
    return audio

def slice_audio_segment(audio: AudioSegment, start: float, end: float) -> AudioSegment:
    """Slice audio segment from start to end (in seconds)."""
    start_ms = max(0, int(start * 1000))
    end_ms = min(len(audio), int(end * 1000))
    return audio[start_ms:end_ms]


# ==========================================================
# Helper: Alignment & WER (Fixed jiwer usage)
# ==========================================================
def levenshtein_alignment(ref: str, hyp: str) -> Dict[str, Any]:
    """Compute WER and alignment statistics using jiwer."""
    try:
        # Clean text first
        ref = ref.strip() if ref else ""
        hyp = hyp.strip() if hyp else ""

        if not ref and not hyp:
            return {"wer": 0.0, "S": 0, "D": 0, "I": 0, "N": 0}
        elif not ref:
            return {"wer": 1.0, "S": 0, "D": 0, "I": len(hyp.split()), "N": 0}
        elif not hyp:
            return {"wer": 1.0, "S": 0, "D": len(ref.split()), "I": 0, "N": len(ref.split())}

        # Use the correct jiwer API
        wer_score = jiwer.wer(ref, hyp)

        # Get detailed measures using jiwer process_words
        measures = jiwer.process_words(ref, hyp)

        return {
            "wer": wer_score,
            "S": measures.substitutions,
            "D": measures.deletions,
            "I": measures.insertions,
            "N": measures.hits + measures.substitutions + measures.deletions,
        }
    except Exception as e:
        print(f"Error in WER computation for ref='{ref}', hyp='{hyp}': {e}")
        return {
            "wer": 1.0,
            "S": 0,
            "D": len(ref.split()) if ref else 0,
            "I": len(hyp.split()) if hyp else 0,
            "N": len(ref.split()) if ref else 0,
        }


# ==========================================================
# Helper function for safe metadata copying
# ==========================================================
def safe_copy_metadata(df_segments: pd.DataFrame, row: pd.Series, original_columns: List[str]) -> pd.DataFrame:
    """Safely copy metadata from original row to segments DataFrame."""
    if df_segments.empty:
        return df_segments

    for col in original_columns:
        if col not in df_segments.columns:
            try:
                value = row[col]
                # Handle different data types safely
                if isinstance(value, (list, np.ndarray)):
                    # Convert to string representation for complex data
                    value = str(value)
                elif pd.isna(value):
                    value = None

                # Assign scalar value to all rows
                df_segments[col] = value

            except Exception as e:
                print(f"Warning: Could not copy column '{col}': {e}")
                df_segments[col] = None

    return df_segments


# ==========================================================
# Thread-safe Whisper Model Manager
# ==========================================================
class WhisperModelManager:
    """Thread-safe manager for Whisper models."""

    def __init__(self, model_size="medium", device="cpu", max_workers=4):
        self.model_size = model_size
        self.device = device
        self.max_workers = max_workers
        self._models = {}
        self._lock = threading.Lock()

    def get_model(self, thread_id: int) -> WhisperModel:
        """Get or create a model for the current thread."""
        with self._lock:
            if thread_id not in self._models:
                model = WhisperModel(self.model_size, device=self.device)
                self._models[thread_id] = model

            return self._models[thread_id]


# ==========================================================
# Processor class with multi-threading support
# ==========================================================
class ASRProcessor:
    def __init__(self, model_size="medium", device="cpu", max_workers=4):
        self.model_manager = WhisperModelManager(model_size, device, max_workers)
        self.max_workers = max_workers

    def transcribe_segment(self, audio: AudioSegment, thread_id: int, batch_mode = True) -> str:
        """Transcribe a single audio segment."""
        model = self.model_manager.get_model(thread_id)
        if batch_mode:
            model = BatchedInferencePipeline(model = model)
        # Create temporary file for this segment
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
            tmp_path = tmp_file.name

        try:
            # Export audio segment to temporary file
            audio.export(tmp_path, format="wav")

            # Transcribe using Whisper
            if not batch_mode:
                segments, _ = model.transcribe(tmp_path)
            else:
                segments, _ = model.transcribe(tmp_path, batch_size = 128)
            text = " ".join([seg.text for seg in segments]).strip()

            return text
        except Exception as e:
            print(f"Error transcribing segment: {e}")
            return ""
        finally:
            # Clean up temporary file
            if os.path.exists(tmp_path):
                os.unlink(tmp_path)

    def process_segment(
        self,
        args: Tuple[AudioSegment, Dict[str, float], int, Optional[str], int]
    ) -> Dict[str, Any]:
        """Process a single segment (for threading)."""
        audio_seg, time_range, idx, reference_text, thread_id = args

        # Slice audio
        seg = slice_audio_segment(audio_seg, time_range["start"], time_range["end"])

        # Check if segment is too short
        if len(seg) < 100:  # Less than 0.1 second
            return {
                "segment_index": idx,
                "start": time_range["start"],
                "end": time_range["end"],
                "recognized_text": "",
                "reference_text": reference_text if reference_text else "",
                "wer": 1.0 if reference_text else 0.0,
                "S": 0,
                "D": len(reference_text.split()) if reference_text else 0,
                "I": 0,
                "N": len(reference_text.split()) if reference_text else 0,
            }

        # Transcribe
        hyp = self.transcribe_segment(seg, thread_id)

        # Prepare result
        result = {
            "segment_index": idx,
            "start": time_range["start"],
            "end": time_range["end"],
            "recognized_text": hyp,
        }

        # Compute WER if reference text is provided
        if reference_text:
            align = levenshtein_alignment(reference_text, hyp)
            result.update({
                "reference_text": reference_text,
                "wer": align["wer"],
                "S": align["S"],
                "D": align["D"],
                "I": align["I"],
                "N": align["N"],
            })

        return result

    def process_file(
        self,
        audio_path: str,
        sr: Optional[int],
        vector: List[int],
        frame_dur: float,
        hop_dur: float,
        pad: int,
        reference_texts: Optional[str] = None,
        show_progress: bool = True,
        fallback_on_no_segments: bool = True,
    ) -> pd.DataFrame:
        """
        Process one audio file: slice segments by vector, run ASR with multi-threading, compute WER.
        If fallback_on_no_segments=True, will create a fallback result with empty text when no segments found.
        """
        try:
            # Load audio
            audio_seg = load_audio_as_pydub(audio_path, sr)
        except Exception as e:
            print(f"Error loading audio {audio_path}: {e}")
            # Return fallback result regardless of reference text
            if fallback_on_no_segments:
                ref_text = reference_texts if reference_texts else ""
                return self._create_fallback_result(ref_text, "file_load_error")
            return pd.DataFrame()

        time_ranges = vector_to_time_ranges(vector, frame_dur, hop_dur, pad)

        # If no segments found, create fallback result
        if not time_ranges:
            print(f"No active segments found for {audio_path}")
            if fallback_on_no_segments:
                # Create fallback even if no reference text
                ref_text = reference_texts if reference_texts else ""
                return self._create_fallback_result(ref_text, "no_segments")
            return pd.DataFrame()

        # Prepare arguments for threading
        args_list = [
            (audio_seg, time_range, idx, reference_texts, idx % self.max_workers)
            for idx, time_range in enumerate(time_ranges)
        ]

        results = []

        # Process segments with threading
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # Submit all tasks
            future_to_args = {
                executor.submit(self.process_segment, args): args
                for args in args_list
            }

            # Collect results with progress bar
            if show_progress:
                pbar = tqdm(total=len(args_list), desc=f"Processing segments from {os.path.basename(audio_path)}")

            for future in as_completed(future_to_args):
                try:
                    result = future.result()
                    results.append(result)
                    if show_progress:
                        pbar.update(1)
                except Exception as e:
                    print(f"Error processing segment: {e}")
                    if show_progress:
                        pbar.update(1)

            if show_progress:
                pbar.close()

        # Sort results by segment index
        results.sort(key=lambda x: x["segment_index"])

        # If no valid segments processed, create fallback result
        if not results:
            print(f"No valid segments processed for {audio_path}")
            if fallback_on_no_segments:
                # Create fallback even if no reference text
                ref_text = reference_texts if reference_texts else ""
                return self._create_fallback_result(ref_text, "processing_failed")
            return pd.DataFrame()

        return pd.DataFrame(results)

    def _create_fallback_result(self, reference_text: str, reason: str = "no_segments") -> pd.DataFrame:
        """
        Create a fallback result when no segments are found but we still want to compute WER.
        Assumes empty hypothesis (no recognized text).

        Logic:
        - If reference is empty/null and hypothesis is empty -> WER = 0 (perfect match)
        - If reference has content and hypothesis is empty -> WER = 1 (complete failure)
        """
        # Clean reference text
        ref_clean = reference_text.strip() if reference_text else ""

        # Compute WER with empty hypothesis
        align = levenshtein_alignment(ref_clean, "")

        result = {
            "segment_index": 0,
            "start": 0.0,
            "end": 0.0,
            "recognized_text": "",
            "reference_text": ref_clean,
            "wer": align["wer"],  # Will be 0 if both ref and hyp are empty
            "S": align["S"],
            "D": align["D"],
            "I": align["I"],
            "N": align["N"],
            "fallback_reason": reason  # Track why this was a fallback
        }

        return pd.DataFrame([result])


# ==========================================================
# Safe parsing function
# ==========================================================
def safe_parse_predictions(preds_str: str) -> Optional[List[int]]:
    """Safely parse predictions string to list."""
    try:
        if isinstance(preds_str, list):
            return preds_str
        if isinstance(preds_str, str):
            # Try ast.literal_eval first (safer than eval)
            try:
                return ast.literal_eval(preds_str)
            except:
                # Fallback to eval if needed
                return eval(preds_str)
        return None
    except Exception as e:
        print(f"Error parsing predictions: {e}")
        return None


# ==========================================================
# Process DataFrame template with multi-threading
# ==========================================================
def process_from_template_df(
    df: pd.DataFrame,
    model_size: str = "medium",
    device: str = "cpu",
    frame_dur: float = 0.02,
    hop_dur: float = 0.01,
    pad: int = 2,
    max_workers: int = 4,
    audio_root_path: str = "/kaggle/input/ddss-aug-ratio-test/test",
    fallback_on_no_segments: bool = True
) -> pd.DataFrame:
    """
    Process dataset from a template DataFrame with columns:
      - id_x: unique id (map to audio path)
      - id_f: folder name
      - preds: binary vector
      - text_active: reference text for WER

    Returns: DataFrame with per-segment evaluation metrics, merged with original metadata.
    """
    print(f"Initializing ASR Processor with model_size={model_size}, device={device}, max_workers={max_workers}")
    processor = ASRProcessor(model_size=model_size, device=device, max_workers=max_workers)
    all_results = []

    print(f"Processing {len(df)} files...")

    # Process each row with progress bar
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing files"):
        try:
            # Resolve audio path
            audio_id = row["id_x"]
            folder_name = row.get("id_f", "")
            audio_path = os.path.join(audio_root_path, folder_name, f"{audio_id}.wav")

            if not os.path.exists(audio_path):
                print(f"Warning: Audio file not found: {audio_path}")
                continue

            # Parse predictions vector safely
            preds = safe_parse_predictions(row["preds"])
            if preds is None:
                print(f"Error parsing predictions for {audio_id}")
                continue

            # Get reference text (can be None/empty)
            ref_text = row.get("text_active", None)
            if pd.isna(ref_text):
                ref_text = None

            # Process file with ASR + WER
            df_segments = processor.process_file(
                audio_path=audio_path,
                sr=row.get("sampling_rate", None),
                vector=preds,
                frame_dur=frame_dur,
                hop_dur=hop_dur,
                pad=pad,
                reference_texts=ref_text,
                show_progress=False,  # Don't show segment progress for each file
                fallback_on_no_segments=fallback_on_no_segments
            )

            if df_segments.empty:
                print(f"No segments processed for {audio_id}")
                continue

            # Copy original metadata to all segments using safe function
            df_segments = safe_copy_metadata(df_segments, row, df.columns.tolist())

            all_results.append(df_segments)

        except Exception as e:
            print(f"Error processing row {idx} (id={row.get('id_x', 'unknown')}): {e}")
            continue

    if not all_results:
        print("Warning: No results to concatenate")
        return pd.DataFrame()

    print(f"Concatenating results from {len(all_results)} processed files...")
    final_df = pd.concat(all_results, ignore_index=True)
    print(f"Final dataset shape: {final_df.shape}")

    return final_df


# ==========================================================
# Utility functions for analysis
# ==========================================================
def compute_summary_stats(df: pd.DataFrame) -> Dict[str, Any]:
    """Compute summary statistics from results DataFrame."""
    if df.empty or 'wer' not in df.columns:
        return {}

    # Filter out invalid WER values
    valid_wer = df[df['wer'].notna() & (df['wer'] >= 0)]

    if valid_wer.empty:
        return {"message": "No valid WER scores found"}

    # Count fallback results
    fallback_count = len(df[df.get('fallback_reason', '').notna()]) if 'fallback_reason' in df.columns else 0
    normal_segments = len(df) - fallback_count

    stats = {
        "total_segments": len(df),
        "normal_segments": normal_segments,
        "fallback_segments": fallback_count,
        "valid_segments": len(valid_wer),
        "mean_wer": valid_wer["wer"].mean(),
        "median_wer": valid_wer["wer"].median(),
        "std_wer": valid_wer["wer"].std(),
        "min_wer": valid_wer["wer"].min(),
        "max_wer": valid_wer["wer"].max(),
        "total_substitutions": df["S"].sum() if "S" in df.columns else 0,
        "total_deletions": df["D"].sum() if "D" in df.columns else 0,
        "total_insertions": df["I"].sum() if "I" in df.columns else 0,
        "total_words": df["N"].sum() if "N" in df.columns else 0,
    }

    # Add fallback breakdown if there are fallback results
    if fallback_count > 0 and 'fallback_reason' in df.columns:
        fallback_reasons = df[df['fallback_reason'].notna()]['fallback_reason'].value_counts().to_dict()
        stats["fallback_breakdown"] = fallback_reasons

    return stats


def save_results(df: pd.DataFrame, output_path: str, include_summary: bool = True):
    """Save results to file with optional summary statistics."""
    if df.empty:
        print("Warning: DataFrame is empty, nothing to save")
        return

    df.to_csv(output_path, index=False)
    print(f"Results saved to: {output_path}")

    if include_summary:
        summary = compute_summary_stats(df)
        if summary:
            summary_path = output_path.replace('.csv', '_summary.txt')
            with open(summary_path, 'w', encoding='utf-8') as f:
                f.write("ASR Evaluation Summary\n")
                f.write("=" * 25 + "\n")
                for key, value in summary.items():
                    f.write(f"{key}: {value}\n")
            print(f"Summary statistics saved to: {summary_path}")


# ==========================================================
# Main execution
# ==========================================================
if __name__ == "__main__":
    # Assume 'merge' DataFrame exists and has the required columns
    # You would need to define or load your 'merge' DataFrame here

    try:
        # Process with multi-threading
        print("Starting ASR evaluation...")
        results_df = process_from_template_df(
            df=merge,  # Make sure this variable is defined
            model_size="base",  # Use smaller model for faster processing
            device="cuda",
            max_workers=4,
            audio_root_path="/kaggle/input/ddss-aug-ratio-test/test",
            fallback_on_no_segments=True  # Enable fallback for files with no segments
        )

        # Save results
        if not results_df.empty:
            save_results(results_df, "asr_evaluation_results.csv")

            # Print summary
            summary = compute_summary_stats(results_df)
            print("\nSummary Statistics:")
            print("=" * 20)
            for key, value in summary.items():
                print(f"{key}: {value}")
        else:
            print("No results to save.")

    except NameError as e:
        print(f"Error: {e}")
        print("Make sure the 'merge' DataFrame is defined before running this script.")
    except Exception as e:
        print(f"Unexpected error: {e}")
        import traceback
        traceback.print_exc()

Starting ASR evaluation...
Initializing ASR Processor with model_size=base, device=cuda, max_workers=4
Processing 15834 files...


Processing files:   0%|          | 0/15834 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/145M [00:00<?, ?B/s]

Processing files:   0%|          | 24/15834 [00:16<2:59:12,  1.47it/s]


KeyboardInterrupt: 